In [ ]:
from audiocraft.models import MusicGen
import torch
import numpy as np
from scipy.io import wavfile
from tqdm import trange
import os
import torchaudio

In [1]:
my_trained_model = MusicGen.get_pretrained('facebook/musicgen-stereo-small')

NameError: name 'MusicGen' is not defined

In [ ]:
my_trained_model.lm.load_state_dict(torch.load('my_save_path/lm_final.pt'))

In [ ]:
my_trained_model.lm = my_trained_model.lm.to(torch.float32)

In [ ]:
my_trained_model.lm.eval()

In [ ]:
# Should be the file used as input into the model
# The model takes mono and outputs stereo
# so this should be the mono "original" audio
myfile = 'model1_data_modified/30S_FreqSweep_Original_0.wav'
s0, bd = wavfile.read(myfile)
# The waveform has to have two channels 
# so we duplicate the 1 channel into both
bd = np.column_stack((bd, bd))
# Convert bd to a tensor
bd_tensor = torch.from_numpy(bd)
bd_tensor = bd_tensor.to(torch.float32)
bd = np.transpose(bd)

In [ ]:
bd_tensor = bd_tensor.unsqueeze(0) # add batch dimension
# 90 in this case is the angle
attributes, prompt_tokens = my_trained_model._prepare_tokens_and_attributes(descriptions=['90'], prompt=bd_tensor)

In [ ]:
# How much to generate, but it accounts for the length of the prompt
max_gen_len = int(30 * my_trained_model.frame_rate) + prompt_tokens.shape[2]

In [ ]:
my_trained_model.generation_params = {
    'max_gen_len': max_gen_len,
    'use_sampling': True,
    'temp': 1.0,
    'top_k': 250,
    'top_p': 0.0,
    'cfg_coef': 3.0,
    'two_step_cfg': 0,
}

In [ ]:
total = []
sample_loops = 4
for _ in trange(sample_loops):
    with my_trained_model.autocast:
        gen_tokens = my_trained_model.lm.generate(prompt_tokens, attributes, callback=None, **my_trained_model.generation_params)
        total.append(gen_tokens[..., prompt_tokens.shape[-1] if prompt_tokens is not None else 0:])
        prompt_tokens = gen_tokens[..., -gen_tokens.shape[-1] // 2:]


In [ ]:
# expands total into what is generated, not really sure how
gen_tokens = torch.cat(total, -1)

# Decode the tokens into audio
with torch.no_grad():
    gen_audio = my_trained_model.compression_model.decode(gen_tokens, None)

In [ ]:
# Saves it
gen_audio = gen_audio.cpu()
save_path = 'my_save_path/'
save_file_name = 'output_audio.wav'
save_path_temp = os.path.join(save_path, save_file_name)
#print(save_path_temp)
torchaudio.save(save_path_temp, gen_audio[0], my_trained_model.sample_rate)